# Assignment: Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`
### Do Q1 and Q2, and either Q3 or Q4, for a total of 3 questions.

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? 
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [5]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 

In [20]:
df = pd.read_csv('./data/airbnb_hw.csv',low_memory=False)

var = 'Price' # clean the Price variable
df[var] = df[var].str.replace("$", "") # removing $
df[var] = df[var].str.replace(",", "") # remove ,
df[var] = df[var].str.strip() # remove leading and trailing white space
df[var] = df[var].replace(["", " ", "0"], np.nan) # replace empty strings, white spaces, and 0 with NaN
print('Before coercion: \n', df[var].describe(),'\n') # The Price variable is read as an object instead of a number

# coerce Price to numeric
df[var] = pd.to_numeric(df[var], errors='coerce')
# Create a missing dummy:
df['Price_nan'] = df[var].isnull() # Equals 1 if missing, 0 if non-null
# After coercion:
print('After coercion: \n', df[var].describe(),'\n') # Describe the numeric variable
print('Total Missings: \n', sum(df['Price_nan']),'\n') # How many missing values are there?


Before coercion: 
 count     30478
unique      511
top         150
freq       1481
Name: Price, dtype: object 

After coercion: 
 count    30478.000000
mean       163.589737
std        197.785454
min         10.000000
25%         80.000000
50%        125.000000
75%        195.000000
max      10000.000000
Name: Price, dtype: float64 

Total Missings: 
 0 



There are 0 missing values. 

In [ ]:
df1 = pd.read_csv('./data/mn_police_use_of_force.csv',low_memory=False)

var = 'subject_injury'
print(df1[var].unique(), '\n') # no empty string so no need for replacement
print(df1[var].value_counts(), '\n')

missing_proportion = df1[var].isna().mean()
print(missing_proportion) # yes. over 75% of values are missing which is an extremely significant portion of data missing

cross_tab = pd.crosstab(df1[var], df1['force_type'])
print(cross_tab)

cross_tab_missing = pd.crosstab(df1[var].isna(), df1['force_type'])
print(cross_tab_missing) 
# majority of missing values come from bodily force, then chemical irritant, then taser
# missing values appear significantly more frequently for these 3 than any other force type

[nan 'No' 'Yes'] 

subject_injury
Yes    1631
No     1446
Name: count, dtype: int64 

0.7619342359767892
force_type      Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury                                                    
No                  0          1093                131        2   
Yes                 2          1286                 41        0   

force_type      Gun Point Display  Improvised Weapon  Less Lethal Projectile  \
subject_injury                                                                 
No                             33                 34                       1   
Yes                            44                 40                       2   

force_type      Police K9 Bite  Taser  
subject_injury                         
No                           2    150  
Yes                         44    172  
force_type      Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury                                                    
False      

In [34]:
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df2 = pd.read_csv(url,low_memory=False)

In [35]:
var = 'WhetherDefendantWasReleasedPretrial'
print(df2[var].unique(), '\n') 
df2[var] = df2[var].replace(9, np.nan) # replace all 9's w/ nan
print(df2[var].value_counts(), '\n')

[9 0 1] 

WhetherDefendantWasReleasedPretrial
1.0    19154
0.0     3801
Name: count, dtype: int64 



In [40]:
var = 'ImposedSentenceAllChargeInContactEvent'

df2[var] = df2[var].str.replace(",", "") # remove ,
df2[var] = df2[var].str.strip() # remove leading and trailing white space
df2[var] = df2[var].replace(["", " "], np.nan) # replace empty strings, white spaces with NaN
print('Before coercion: \n', df2[var].describe(),'\n') # The variable is read as an object instead of a number

# coerce Price to numeric
df2[var] = pd.to_numeric(df2[var], errors='coerce')
# Create a missing dummy:
df2['ImposedSentence_nan'] = df2[var].isnull() # Equals 1 if missing, 0 if non-null
# After coercion:
print('After coercion: \n', df2[var].describe(),'\n') # Describe the numeric variable
print('Total Missings: \n', sum(df2['ImposedSentence_nan']),'\n') 

AttributeError: Can only use .str accessor with string values!

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q4.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?